In [ ]:
import os
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Bidirectional, GRU, Dense, Dropout, Input, Layer

# ───────────────────────────────────────────────────────────────
# CONFIGURACIÓN
# ───────────────────────────────────────────────────────────────

BASE_PATH = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\Conjunt de dades Preprocessades\Datasets"
RESULTS_PATH = "resultats_GRU_Attention"
DATASETS     = [
    "Amazon_Stock_Price_output.csv",
    "Euro_Stoxx_50_Stock_Price_output.csv",
    "Google_Stock_Price_output.csv",
    "Hang_Seng_Stock_Price_output.csv",
    "IBEX_35_Stock_Price_output.csv",
    "Indra_Stock_Price_output.csv",
    "P&G_Stock_Price_output.csv",
    "S&P500_Stock_Price_output.csv"
]

N_STEPS = 20
FEATURE_COLUMNS = [
    "Open","High","Low","Volume",
    "EMA_7","EMA_40","MACD","Signal_Line",
    "MACD_Hist","RSI","ATR"
]
TARGET_COLUMN = "Close"
TEST_RATIO = 0.10
VAL_RATIO  = 0.10
TRAIN_RATIO = 1 - TEST_RATIO - VAL_RATIO

# ───────────────────────────────────────────────────────────────
# FUNCIONES AUXILIARES
# ───────────────────────────────────────────────────────────────

def create_sequences(X, y, n_steps=20):
    Xs, ys = [], []
    for i in range(n_steps, len(X)):
        Xs.append(X[i - n_steps:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

def compute_metrics(model, X_scaled, y_scaled, scaler_y):
    """
    Calcula MAE, RMSE, R² y devuelve y_true, y_pred.
    """
    y_pred_scaled = model.predict(X_scaled, verbose=0)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_scaled)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2, y_true, y_pred

# ───────────────────────────────────────────────────────────────
# Capa de atención (igual que en entrenamiento)
# ───────────────────────────────────────────────────────────────

class Attention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(name="attn_W",
                                 shape=(input_shape[-1], 1),
                                 initializer="glorot_uniform",
                                 trainable=True)
        super().build(input_shape)
    def call(self, inputs):
        # inputs: (batch, timesteps, features)
        score   = tf.matmul(tf.tanh(inputs), self.W)   # (batch, timesteps, 1)
        weights = tf.nn.softmax(score, axis=1)          # (batch, timesteps, 1)
        context = tf.reduce_sum(weights * inputs, axis=1)  # (batch, features)
        return context

# ───────────────────────────────────────────────────────────────
# Reconstruir el modelo GRU+Attention con hiperparámetros cargados
# ───────────────────────────────────────────────────────────────

def build_gru_attention_model(look_back, n_features, params):
    """
    Reconstruye la arquitectura de GRU+Attention con los hiperparámetros dados:
      - conv_filters: número de filtros en Conv1D
      - kernel_size: tamaño del kernel de Conv1D
      - gru_units: unidades GRU
      - dropout_rate: tasa de dropout
      - dense_units: unidades de la capa densa
      - learning_rate
      - l2_reg: factor de regularización L2
    """
    model = Sequential([
        Input(shape=(look_back, n_features)),
        Conv1D(filters=params['conv_filters'],
               kernel_size=params['kernel_size'],
               activation='relu',
               padding='same'),
        MaxPooling1D(pool_size=2),
        Bidirectional(GRU(units=params['gru_units'],
                          return_sequences=True,
                          recurrent_dropout=params['dropout_rate'],
                          kernel_regularizer=tf.keras.regularizers.l2(params['l2_reg'])
                         )),
        Attention(),
        Dropout(params['dropout_rate']),
        Dense(params['dense_units'],
              activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(params['l2_reg'])),
        Dropout(params['dropout_rate']),
        Dense(1, activation='linear')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# ───────────────────────────────────────────────────────────────
# BUCLE PRINCIPAL: CARGAR CADA DATASET, LEER PESOS E HIPERPARÁMETROS,
# CALCULAR MÉTRICAS Y GRAFICAR
# ───────────────────────────────────────────────────────────────

for filename in DATASETS:
    # Obtener nombre base en minúsculas para carpeta de resultados
    dataset_name_base = os.path.splitext(filename)[0].lower()
    print(f"\n===== Procesando: {dataset_name_base} =====")

    # Rutas
    data_path    = os.path.join(BASE_PATH, filename)
    model_folder = os.path.join(RESULTS_PATH, dataset_name_base)
    weights_path = os.path.join(model_folder, "best_model_weights.weights.h5")
    params_path  = os.path.join(model_folder, "best_hyperparameters.json")

    if not os.path.isfile(weights_path) or not os.path.isfile(params_path):
        print(f"  ⚠️  Faltan pesos o parámetros para {dataset_name_base}, omitiendo.")
        continue

    # 1) Cargar CSV original y recalcular indicadores
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values("Date", inplace=True)
    df = df.reset_index(drop=True)
    # Recalcular indicadores clave
    df['EMA_7']   = df['Close'].ewm(span=7, adjust=False).mean()
    df['EMA_40']  = df['Close'].ewm(span=40, adjust=False).mean()
    ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD']        = ema_12 - ema_26
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist']   = df['MACD'] - df['Signal_Line']
    delta = df['Close'].diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-8)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['ATR'] = (df['High'] - df['Low']).rolling(window=14).mean()

    df = df.dropna(subset=FEATURE_COLUMNS + [TARGET_COLUMN]).reset_index(drop=True)

    # 2) Crear secuencias y dividir en train/val/test
    data_X = df[FEATURE_COLUMNS].values
    data_y = df[TARGET_COLUMN].values.reshape(-1, 1)
    X_seq, y_seq = create_sequences(data_X, data_y, n_steps=N_STEPS)

    n_total   = len(X_seq)
    train_end = int(n_total * TRAIN_RATIO)
    val_end   = train_end + int(n_total * VAL_RATIO)

    X_train = X_seq[:train_end]
    y_train = y_seq[:train_end]

    X_val   = X_seq[train_end:val_end]
    y_val   = y_seq[train_end:val_end]

    X_test  = X_seq[val_end:]
    y_test  = y_seq[val_end:]

    # 3) Reconstruir escaladores con train+val (igual que en entrenamiento)
    scaler_X = StandardScaler()
    X_train_val_flat = np.vstack((X_train, X_val)).reshape(-1, len(FEATURE_COLUMNS))
    scaler_X.fit(X_train_val_flat)

    scaler_y = StandardScaler()
    y_train_val_flat = np.vstack((y_train, y_val))
    scaler_y.fit(y_train_val_flat)

    def scale_X(X):
        flat = X.reshape(-1, len(FEATURE_COLUMNS))
        flat_scaled = scaler_X.transform(flat)
        return flat_scaled.reshape(X.shape)

    X_train_scaled = scale_X(X_train)
    X_val_scaled   = scale_X(X_val)
    X_test_scaled  = scale_X(X_test)

    y_train_scaled = scaler_y.transform(y_train)
    y_val_scaled   = scaler_y.transform(y_val)
    y_test_scaled  = scaler_y.transform(y_test)

    # 4) Cargar hiperparámetros
    with open(params_path, 'r') as f:
        best_params = json.load(f)
    print(f"  ✓ Parámetros cargados: {best_params}")

    # 5) Reconstruir modelo y cargar pesos
    model = build_gru_attention_model(
        look_back=N_STEPS,
        n_features=len(FEATURE_COLUMNS),
        params=best_params
    )
    model.load_weights(weights_path)
    print(f"  ✓ Pesos cargados desde: {weights_path}")

    # 6) Calcular métricas en Validation y Test
    mae_val, rmse_val, r2_val, y_val_true, y_val_pred = compute_metrics(
        model, X_val_scaled, y_val_scaled, scaler_y
    )
    mae_test, rmse_test, r2_test, y_test_true, y_test_pred = compute_metrics(
        model, X_test_scaled, y_test_scaled, scaler_y
    )
    print(f"  → Val MAE={mae_val:.4f}, RMSE={rmse_val:.4f}, R²={r2_val:.4f}")
    print(f"  → Test MAE={mae_test:.4f}, RMSE={rmse_test:.4f}, R²={r2_test:.4f}")

    # 6.1) Guardar métricas finales en CSV (siguiendo convención metrics_summary.csv)
    df_metrics = pd.DataFrame({
        "Dataset":   [dataset_name_base],
        "Val_MAE":   [mae_val],
        "Val_RMSE":  [rmse_val],
        "Val_R2":    [r2_val],
        "Test_MAE":  [mae_test],
        "Test_RMSE": [rmse_test],
        "Test_R2":   [r2_test]
    })
    metrics_csv = os.path.join(model_folder, "metrics_summary.csv")
    df_metrics.to_csv(metrics_csv, index=False)
    print(f"  ✓ Métricas guardadas en: {metrics_csv}")

    # 7) Graficar Real vs Predicho (Test)
    start_test_idx = N_STEPS + val_end
    dates_test = df['Date'].iloc[start_test_idx : start_test_idx + len(y_test_true)].reset_index(drop=True)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_test,
        y=y_test_true.flatten(),
        mode='lines',
        name='Real (Close)',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=dates_test,
        y=y_test_pred.flatten(),
        mode='lines',
        name='Predicho',
        line=dict(color='red', dash='dash')
    ))
    fig.update_layout(
        title=f"{dataset_name_base} – Real vs Predicción (Test)",
        xaxis_title='Fecha',
        yaxis_title='Precio Close (USD)',
        template='plotly_white',
        xaxis_rangeslider_visible=True
    )
    plot_html = os.path.join(model_folder, f"{dataset_name_base}_test_plot.html")
    fig.write_html(plot_html)
    print(f"  ✓ Gráfica Test guardada en: {plot_html}")

    # 8) Predicción autoregresiva de los próximos 10 días laborables
    df_future = df.copy().reset_index(drop=True)
    # Recalcular indicadores en todo df_future
    df_future['EMA_7']   = df_future['Close'].ewm(span=7, adjust=False).mean()
    df_future['EMA_40']  = df_future['Close'].ewm(span=40, adjust=False).mean()
    ema_12 = df_future['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = df_future['Close'].ewm(span=26, adjust=False).mean()
    df_future['MACD']        = ema_12 - ema_26
    df_future['Signal_Line'] = df_future['MACD'].ewm(span=9, adjust=False).mean()
    df_future['MACD_Hist']   = df_future['MACD'] - df_future['Signal_Line']
    delta = df_future['Close'].diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-8)
    df_future['RSI'] = 100 - (100 / (1 + rs))
    df_future['ATR'] = (df_future['High'] - df_future['Low']).rolling(window=14).mean()

    last_sequence = X_test_scaled[-1].copy().reshape(1, N_STEPS, len(FEATURE_COLUMNS))
    future_preds = []
    future_dates = pd.bdate_range(start=df_future['Date'].iloc[-1] + pd.Timedelta(days=1), periods=10)

    for date in future_dates:
        # 8.1) Predicción escalada y remapeo a valor real
        y_pred_scaled = model.predict(last_sequence, verbose=0)[0][0]
        y_pred_real = scaler_y.inverse_transform([[y_pred_scaled]])[0][0]
        future_preds.append(y_pred_real)

        # 8.2) Añadir la predicción al df_future para recalcular indicadores
        prev = df_future.iloc[-1]
        new_row = {
            'Date':        date,
            'Open':        prev['Close'],
            'High':        y_pred_real,
            'Low':         y_pred_real,
            'Volume':      prev['Volume'],
            'Close':       y_pred_real,
            'EMA_7':       np.nan,
            'EMA_40':      np.nan,
            'MACD':        np.nan,
            'Signal_Line': np.nan,
            'MACD_Hist':   np.nan,
            'RSI':         np.nan,
            'ATR':         np.nan
        }
        df_future.loc[len(df_future)] = new_row

        # Recalcular indicadores en df_future completo
        df_future['EMA_7']   = df_future['Close'].ewm(span=7, adjust=False).mean()
        df_future['EMA_40']  = df_future['Close'].ewm(span=40, adjust=False).mean()
        ema_12 = df_future['Close'].ewm(span=12, adjust=False).mean()
        ema_26 = df_future['Close'].ewm(span=26, adjust=False).mean()
        df_future['MACD']        = ema_12 - ema_26
        df_future['Signal_Line'] = df_future['MACD'].ewm(span=9, adjust=False).mean()
        df_future['MACD_Hist']   = df_future['MACD'] - df_future['Signal_Line']
        delta = df_future['Close'].diff()
        gain  = delta.clip(lower=0)
        loss  = -delta.clip(upper=0)
        avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
        rs = avg_gain / (avg_loss + 1e-8)
        df_future['RSI'] = 100 - (100 / (1 + rs))
        df_future['ATR'] = (df_future['High'] - df_future['Low']).rolling(window=14).mean()

        # 8.3) Extraer las últimas N_STEPS filas de indicadores técnicos
        recent_features = df_future[FEATURE_COLUMNS].iloc[-N_STEPS:].values
        recent_scaled = scaler_X.transform(recent_features)
        last_sequence = recent_scaled.reshape(1, N_STEPS, len(FEATURE_COLUMNS))

    # 9) Guardar predicciones futuras en CSV
    df_fut_pred = pd.DataFrame({
        "Date":            future_dates,
        "Predicted_Close": future_preds
    })
    fut_csv = os.path.join(model_folder, "future_10days.csv")
    df_fut_pred.to_csv(fut_csv, index=False)
    print(f"  ✓ Predicciones futuras guardadas en: {fut_csv}")

    # 10) Graficar histórico + predicciones futuras
    fig_future = go.Figure()
    fig_future.add_trace(go.Scatter(
        x=df['Date'], y=df['Close'],
        mode='lines', name='Histórico Close', line=dict(color='lightblue')
    ))
    fig_future.add_trace(go.Scatter(
        x=future_dates, y=np.array(future_preds),
        mode='lines+markers', name='Predicción futura',
        line=dict(color='orange', dash='dash'),
        marker=dict(size=6)
    ))
    fig_future.update_layout(
        title=f"{dataset_name_base} – Predicción Próximos 10 Días",
        xaxis_title='Fecha',
        yaxis_title='Precio Close (USD)',
        template='plotly_white',
        xaxis_rangeslider_visible=True
    )
    fut_html = os.path.join(model_folder, "future_plot.html")
    fig_future.write_html(fut_html)
    print(f"  ✓ Gráfica futura guardada en: {fut_html}")



===== Procesando: amazon_stock_price_output =====
  ✓ Parámetros cargados: {'conv_filters': 64, 'kernel_size': 3, 'gru_units': 32, 'dropout_rate': 0.1, 'l2_reg': 0.001, 'dense_units': 32, 'learning_rate': 0.004969724253724245}



c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 28 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


  ✓ Pesos cargados desde: resultats_GRU_Attention\amazon_stock_price_output\best_model_weights.weights.h5
  → Val MAE=3.1704, RMSE=4.0301, R²=0.7074
  → Test MAE=10.9723, RMSE=14.0063, R²=0.5735
  ✓ Métricas guardadas en: resultats_GRU_Attention\amazon_stock_price_output\metrics_summary.csv
  ✓ Gráfica Test guardada en: resultats_GRU_Attention\amazon_stock_price_output\amazon_stock_price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: resultats_GRU_Attention\amazon_stock_price_output\future_10days.csv
  ✓ Gráfica futura guardada en: resultats_GRU_Attention\amazon_stock_price_output\future_plot.html

===== Procesando: euro_stoxx_50_stock_price_output =====
  ✓ Parámetros cargados: {'conv_filters': 32, 'kernel_size': 3, 'gru_units': 128, 'dropout_rate': 0.2, 'l2_reg': 0.0001, 'dense_units': 32, 'learning_rate': 0.0005770793146610146}


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 28 variables. 



  ✓ Pesos cargados desde: resultats_GRU_Attention\euro_stoxx_50_stock_price_output\best_model_weights.weights.h5
  → Val MAE=51.2774, RMSE=62.5857, R²=0.6182
  → Test MAE=39.2247, RMSE=50.9520, R²=0.8599
  ✓ Métricas guardadas en: resultats_GRU_Attention\euro_stoxx_50_stock_price_output\metrics_summary.csv
  ✓ Gráfica Test guardada en: resultats_GRU_Attention\euro_stoxx_50_stock_price_output\euro_stoxx_50_stock_price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: resultats_GRU_Attention\euro_stoxx_50_stock_price_output\future_10days.csv
  ✓ Gráfica futura guardada en: resultats_GRU_Attention\euro_stoxx_50_stock_price_output\future_plot.html

===== Procesando: google_stock_price_output =====
  ✓ Parámetros cargados: {'conv_filters': 32, 'kernel_size': 5, 'gru_units': 64, 'dropout_rate': 0.30000000000000004, 'l2_reg': 0.001, 'dense_units': 64, 'learning_rate': 0.006709934026096912}
  ✓ Pesos cargados desde: resultats_GRU_Attention\google_stock_price_output\best_model_weight

...

  → Val MAE=364.2875, RMSE=478.2583, R²=0.9020
  → Test MAE=370.6862, RMSE=482.7811, R²=0.4030
  ✓ Métricas guardadas en: resultats_GRU_Attention\indra_stock_price_output\metrics_summary.csv
  ✓ Gráfica Test guardada en: resultats_GRU_Attention\indra_stock_price_output\indra_stock_price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: resultats_GRU_Attention\indra_stock_price_output\future_10days.csv
  ✓ Gráfica futura guardada en: resultats_GRU_Attention\indra_stock_price_output\future_plot.html

===== Procesando: p&g_stock_price_output =====
  ✓ Parámetros cargados: {'conv_filters': 32, 'kernel_size': 3, 'gru_units': 128, 'dropout_rate': 0.1, 'l2_reg': 0.001, 'dense_units': 16, 'learning_rate': 0.004568991430598756}
  ✓ Pesos cargados desde: resultats_GRU_Attention\p&g_stock_price_output\best_model_weights.weights.h5


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 28 variables. 



  → Val MAE=1.1489, RMSE=1.5629, R²=0.8465
  → Test MAE=1.7199, RMSE=2.2272, R²=0.7550
  ✓ Métricas guardadas en: resultats_GRU_Attention\p&g_stock_price_output\metrics_summary.csv
  ✓ Gráfica Test guardada en: resultats_GRU_Attention\p&g_stock_price_output\p&g_stock_price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: resultats_GRU_Attention\p&g_stock_price_output\future_10days.csv
  ✓ Gráfica futura guardada en: resultats_GRU_Attention\p&g_stock_price_output\future_plot.html

===== Procesando: s&p500_stock_price_output =====
  ✓ Parámetros cargados: {'conv_filters': 32, 'kernel_size': 3, 'gru_units': 32, 'dropout_rate': 0.1, 'l2_reg': 0.001, 'dense_units': 64, 'learning_rate': 0.007487658008462713}
  ✓ Pesos cargados desde: resultats_GRU_Attention\s&p500_stock_price_output\best_model_weights.weights.h5


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 28 variables. 



  → Val MAE=78.1452, RMSE=90.6843, R²=0.7704
  → Test MAE=148.7868, RMSE=172.7577, R²=0.3705
  ✓ Métricas guardadas en: resultats_GRU_Attention\s&p500_stock_price_output\metrics_summary.csv
  ✓ Gráfica Test guardada en: resultats_GRU_Attention\s&p500_stock_price_output\s&p500_stock_price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: resultats_GRU_Attention\s&p500_stock_price_output\future_10days.csv
  ✓ Gráfica futura guardada en: resultats_GRU_Attention\s&p500_stock_price_output\future_plot.html
